In [1]:
#!pip install -q git+https://github.com/deepmipt/bert.git@feat/keras1 tensorflow-gpu==1.14.0rc1 deeppavlov

In [2]:
import tensorflow as tf
print(tf.__version__)
# tf.compat.v1.enable_eager_execution()
tf.compat.v1.executing_eagerly()

1.14.0-rc1


False

# Paraphrases data

In [3]:
BATCH_SIZE = 64  # both for training and evaluation for now
MAX_LEN = 52  # the longest sequence in the paraphrase dataset, workaroud that make shape of all batch tensors equal
NUM_TRAIN_SAMPLES = 6656  # total of 7202 samples minus ~500 (actually 546) samples for validation
DYNAMIC_SEQ_LEN = True  # whether to pad each batch to the maximum len in the batch, or in the whole dataset
NUM_CLASSES = 2  # binary classification as multiclass
DROP_REMAINDER = True # whether to for batches of fixed size

In [4]:
tf.keras.utils.get_file(
    fname='paraphraser.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser.zip',
    cache_subdir='dataset',
    extract=True,
    cache_dir='.'
)
tf.keras.utils.get_file(
    fname='paraphraser_gold.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser_gold.zip',
    cache_subdir='dataset',
    extract=True,
    archive_format='zip',
    cache_dir='.'
)
tf.keras.utils.get_file(
    fname='rubert_cased_L-12_H-768_A-12_v1_converted.zip',
    origin='http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1_converted.zip',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

'./models/rubert_cased_L-12_H-768_A-12_v1_converted.zip'

In [5]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def data_gen(train_data=True):
    from deeppavlov.dataset_readers.paraphraser_reader import ParaphraserReader
    from bert_dp.tokenization import FullTokenizer
    ds = ParaphraserReader().read(data_path='dataset', do_lower_case=False)
    tokenizer = FullTokenizer(vocab_file='models/rubert_cased_L-12_H-768_A-12_v1_converted/vocab.txt', do_lower_case=False)
    tds = ds['train'] if train_data else ds['test']
    for pair, label in tds:
        t1 = tokenizer.tokenize(pair[0])
        t2 = tokenizer.tokenize(pair[1])
        _truncate_seq_pair(t1, t2, 512)
        s = ['[CLS]'] + t1 + ['[SEP]'] + t2 + ['[SEP]']
        yield tokenizer.convert_tokens_to_ids(s), label

In [6]:
import numpy as np

x_train_, y_train_, x_test_, y_test_ = [], [], [], []
for x, y in data_gen():
    x_train_.append(np.pad(x, pad_width=(0, MAX_LEN - len(x)), mode='constant'))
    y_train_.append(y)
for x, y in data_gen(train_data=False):
    x_test_.append(np.pad(x, pad_width=(0, MAX_LEN - len(x)), mode='constant'))
    y_test_.append(y)
x_train = np.array(x_train_)
y_train = np.array(y_train_)
x_test = np.array(x_test_)
y_test = np.array(y_test_)
len(x_train_), len(x_test_)

W0611 21:08:46.860945 140706646267712 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



(7202, 1899)

In [7]:
# train_ds_ = tf.data.Dataset.from_generator(data_gen,
#                                            output_types=(tf.int32, tf.int32),
#                                            output_shapes=(tf.TensorShape([None]),
#                                                           tf.TensorShape([])))
# train_ds = train_ds_.take(NUM_TRAIN_SAMPLES)
# valid_ds = train_ds_.skip(NUM_TRAIN_SAMPLES)
# batched_train_ds = train_ds.padded_batch(batch_size=BATCH_SIZE,
#                                          padded_shapes=([-1] if DYNAMIC_SEQ_LEN else MAX_LEN,
#                                                         ()),
#                                          drop_remainder=DROP_REMAINDER)
# batched_valid_ds = valid_ds.padded_batch(batch_size=BATCH_SIZE,
#                                          padded_shapes=([-1] if DYNAMIC_SEQ_LEN else MAX_LEN,
#                                                         ()),
#                                          drop_remainder=DROP_REMAINDER)
# test_ds = tf.data.Dataset.from_generator(data_gen,
#                                          output_types=(tf.int32, tf.int32),
#                                          output_shapes=(tf.TensorShape([None]),
#                                                         tf.TensorShape([])),
#                                          args=(False,))
# batched_test_ds = test_ds.padded_batch(batch_size=BATCH_SIZE,
#                                        padded_shapes=([-1] if DYNAMIC_SEQ_LEN else MAX_LEN,
#                                                       ()),
#                                        drop_remainder=DROP_REMAINDER)

# Model construction and compilation for training

In [8]:
from bert_dp import bert
try:
    del paraphrase_detector
    tf.keras.backend.clear_session()
    print('Session cleared')
except NameError:
    pass
paraphrase_detector = tf.keras.Sequential([bert.BERT(name='bert'),
                                           tf.keras.layers.Dropout(rate=0.5),
                                           tf.keras.layers.Dense(NUM_CLASSES)])

W0611 21:09:15.507581 140706646267712 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/embeddings.py:30: The name tf.keras.initializers.TruncatedNormal is deprecated. Please use tf.compat.v1.keras.initializers.TruncatedNormal instead.

W0611 21:09:15.508118 140706646267712 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:94: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0611 21:09:15.515448 140706646267712 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instru

In [9]:
# some data to pass through model in order to construct and autograph it
try:
    for s in batched_train_ds:
        r = paraphrase_detector(s[0])
        print('Model constructed by means of passing one batch from tf.data')
        break
except NameError:
    r = paraphrase_detector.predict(x_train)
    print('Model constructed by means of passing one batch from array slice')
except RuntimeError:
    paraphrase_detector.build(input_shape=(BATCH_SIZE, None if DYNAMIC_SEQ_LEN else MAX_LEN))
    print('Model constructed using .build() method')

Model constructed by means of passing one batch from array slice


In [10]:
from bert_dp.metrics import SparseBinaryF1Score

NUM_EPOCHS = 5
# VAL_EVERY_N_BATCHES = 52  # ~50, but better suited for paraphraser dataset
# TRAIN_STEPS_PER_EPOCH = NUM_TRAIN_SAMPLES // BATCH_SIZE
# VALIDATION_STEPS = 512 // BATCH_SIZE  # calculated by hand for now (~546 samples with droppping remainder)
LEARNING_RATE = 2e-05
# learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=LEARNING_RATE,
#                                                               # n. of batches per epoch * ~NUM_EPOCHS for convergence 
#                                                               decay_steps=TRAIN_STEPS_PER_EPOCH * NUM_EPOCHS,
#                                                               end_learning_rate=1e-06)
# from bert_dp.weight_decay_optimizers import AdamW
paraphrase_detector.compile(#optimizer=AdamW(weight_decay=0.01,
#                                             learning_rate=LEARNING_RATE,
#                                             epsilon=1e-6),  # as in bert_dp
                            optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,
                                                               epsilon=1e-6),
                            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                            metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),
                                     SparseBinaryF1Score()
                                    ])

In [13]:
# baseline to ensure something is learning
# paraphrase_detector.evaluate(batched_test_ds)
paraphrase_detector.evaluate(x=x_test, y=y_test, batch_size=BATCH_SIZE)

1899/1899 [==============================] - 4s 2ms/sample - loss: 0.7348 - sparse_categorical_accuracy: 0.4234 - sparse_binary_f1_score: 0.3573


[0.7348496178817348, 0.42338073, 0.3572744]

In [11]:
# paraphrase_detector.layers[0].load_weights('models/rubert_cased_L-12_H-768_A-12_v1_converted/bert_model').assert_consumed()

In [12]:
paraphrase_detector.load_weights('models/paraphraser_rubert_converted/model_rubert').assert_consumed()

AssertionError: Some Python objects were not bound to checkpointed values, likely due to changes in the Python program: [<tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7ff848ce3940>]

# Training

In [16]:
# baseline to ensure something is learning
# paraphrase_detector.evaluate(batched_test_ds)
paraphrase_detector.evaluate(x=x_test, y=y_test, batch_size=BATCH_SIZE)

1899/1899 [==============================] - 5s 2ms/sample - loss: 0.7434 - sparse_categorical_accuracy: 0.5972 - sparse_binary_f1_score: 0.0510


[0.7434239565792555, 0.5971564, 0.05097706]

In [19]:
# tf.keras.backend.set_learning_phase(1)

In [18]:
history = paraphrase_detector.fit(x=x_train,
                                  y=y_train,
                                  epochs=NUM_EPOCHS,
                                  callbacks=[tf.keras.callbacks.TensorBoard(batch_size=BATCH_SIZE),
                                             tf.keras.callbacks.ReduceLROnPlateau(monitor='val_sparse_binary_f1_score',
                                                                                  factor=0.5,
                                                                                  patience=3,
                                                                                  min_lr=1e-06)],
                                  validation_split=0.071)  # ~512 samples
# history = paraphrase_detector.fit(x=batched_train_ds.repeat(),
#                                   y=None,
#                                   epochs=NUM_EPOCHS * TRAIN_STEPS_PER_EPOCH // VAL_EVERY_N_BATCHES,  # workaround for more frequent evaluation
#                                   callbacks=[tf.keras.callbacks.TensorBoard(batch_size=BATCH_SIZE),
#                                              tf.keras.callbacks.ReduceLROnPlateau(monitor='val_sparse_binary_f1_score',
#                                                                                   factor=0.5,
#                                                                                   patience=3,
#                                                                                   min_lr=1e-06)],
#                                   validation_data=batched_valid_ds,
#                                   steps_per_epoch=VAL_EVERY_N_BATCHES,
#                                   validation_steps=VALIDATION_STEPS)

Train on 6689 samples, validate on 513 samples
Epoch 1/5
6689/6689 [==============================] - 51s 8ms/sample - loss: 0.8750 - sparse_categorical_accuracy: 0.5630 - sparse_binary_f1_score: 0.7033 - val_loss: 0.7937 - val_sparse_categorical_accuracy: 0.5906 - val_sparse_binary_f1_score: 0.7426
Epoch 2/5
6689/6689 [==============================] - 48s 7ms/sample - loss: 0.8355 - sparse_categorical_accuracy: 0.5703 - sparse_binary_f1_score: 0.6713 - val_loss: 0.8103 - val_sparse_categorical_accuracy: 0.4230 - val_sparse_binary_f1_score: 0.7229
Epoch 3/5
6689/6689 [==============================] - 48s 7ms/sample - loss: 0.7224 - sparse_categorical_accuracy: 0.6582 - sparse_binary_f1_score: 0.7389 - val_loss: 0.7031 - val_sparse_categorical_accuracy: 0.5965 - val_sparse_binary_f1_score: 0.7426
Epoch 4/5
6689/6689 [==============================] - 49s 7ms/sample - loss: 0.5389 - sparse_categorical_accuracy: 0.7660 - sparse_binary_f1_score: 0.7590 - val_loss: 0.8632 - val_sparse_cat

In [19]:
# baseline to ensure something is learning
# paraphrase_detector.evaluate(batched_test_ds)
paraphrase_detector.evaluate(x=x_test, y=y_test, batch_size=BATCH_SIZE)

1899/1899 [==============================] - 4s 2ms/sample - loss: 1.1100 - sparse_categorical_accuracy: 0.6114 - sparse_binary_f1_score: 0.7494


[1.1100106747166743, 0.6113744, 0.7494238]